In [1]:
pip install reportlab

     ---------------------------------------- 1.9/1.9 MB 30.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install 

In [2]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
from reportlab.pdfgen import canvas


# Conectar ao banco de dados SQLite

connect = sqlite3.connect('prova_db.db')

# Criar cursor que permite acessar o banco de dados e então percorrê-lo utilizando da linguagem SQL
c = connect.cursor() 

# Criar tabelas (se não existirem)

# Usuários
 
c.execute('''
    CREATE TABLE IF NOT EXISTS usuarios (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT,
        password TEXT,
        admin INTEGER
    )
''')

# Cursos 

c.execute('''
    CREATE TABLE IF NOT EXISTS cursos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT
    )
''')

# Disciplinas

c.execute('''
    CREATE TABLE IF NOT EXISTS disciplinas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        curso_id INTEGER,
        FOREIGN KEY (curso_id) REFERENCES cursos (id)
    )
''')

# Questões

c.execute('''
    CREATE TABLE IF NOT EXISTS questoes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        enunciado TEXT,
        disciplina_id INTEGER,
        FOREIGN KEY (disciplina_id) REFERENCES disciplinas (id)
    )
''')
        
# Função Criar PDF

def criar_pdf(prova):
    filename = "prova.pdf"
    c = canvas.Canvas(filename)     # cria o pdf com o nome
    c.drawString(100, 800, "Cabeçalho da prova")    # drawString(x, y, "texto a ser escrito")
    
    y = 700                                      # posição inicial do cursor (canto superior esquerdo, com distância 100 da margem)
    for i, questao in enumerate(prova, start=1):
        c.drawString(100, y, f"{i}.{questao}")
        y -= 20                                  # reduz a posição do eixo y (vertical) em 20 a cada iteração
    
    c.save()

    
# Função para login

def fazer_login():
    
    username = entry_username.get()   # entry object in tkinter package; .get -> Returns the entry's current text as a string
    password = entry_password.get()
    
    c.execute("SELECT * FROM usuarios WHERE username=? AND password=?", (username, password))
    usuario = c.fetchone()            # .fetchone() -> retrieves the next row of a query result set and returns a single sequence

    if usuario:
        if usuario[3] == 1:  # Verifica se é um administrador
            admin_window(usuario[0])
        else:
            usuario_window(usuario[0])
    else:
        messagebox.showerror("Erro de Login", "Credenciais inválidas")
    
# Janela de Login
def login_window():
    login_window = tk.Tk()       # Provides access to the Tcl interpreter
    login_window.title("Login")

    global entry_username, entry_password                     # turns username and password as global variables
    label_username = tk.Label(login_window, text="Username:") 
    label_password = tk.Label(login_window, text="Password:")
    entry_username = tk.Entry(login_window)
    entry_password = tk.Entry(login_window, show="*")
    button_login = tk.Button(login_window, text="Login", command=fazer_login) # command

    label_username.grid(row=0, column=0)
    entry_username.grid(row=0, column=1)
    label_password.grid(row=1, column=0)
    entry_password.grid(row=1, column=1)
    button_login.grid(row=2, column=1)

    login_window.mainloop()

# Janela para administradores
def admin_window(usuario_id):
    admin_window = tk.Tk()
    admin_window.title("Área do Administrador")

    def cadastrar_curso():
        nome_curso = entry_nome_curso.get()
        c.execute("INSERT INTO cursos (nome) VALUES (?)", (nome_curso,))
        conn.commit()
        messagebox.showinfo("Curso Cadastrado", f"Curso {nome_curso} cadastrado com sucesso!")

    def cadastrar_disciplina():
        nome_disciplina = entry_nome_disciplina.get()
        curso_id = combo_cursos.get()
        c.execute("INSERT INTO disciplinas (nome, curso_id) VALUES (?, ?)", (nome_disciplina, curso_id))
        conn.commit()
        messagebox.showinfo("Disciplina Cadastrada", f"Disciplina {nome_disciplina} cadastrada com sucesso!")

    label_nome_curso = tk.Label(admin_window, text="Nome do Curso:")
    entry_nome_curso = tk.Entry(admin_window)
    button_cadastrar_curso = tk.Button(admin_window, text="Cadastrar Curso", command=cadastrar_curso)

    label_nome_disciplina = tk.Label(admin_window, text="Nome da Disciplina:")
    entry_nome_disciplina = tk.Entry(admin_window)

    c.execute("SELECT * FROM cursos")
    cursos = c.fetchall()
    combo_cursos = tk.StringVar()
    combo_cursos.set(cursos[0][0])  # Definir o valor padrão
    dropdown_cursos = tk.OptionMenu(admin_window, combo_cursos, *cursos)

    button_cadastrar_disciplina = tk.Button(admin_window, text="Cadastrar Disciplina", command=cadastrar_disciplina)

    label_nome_curso.grid(row=0, column=0)
    entry_nome_curso.grid(row=0, column=1)
    button_cadastrar_curso.grid(row=0, column=2)

    label_nome_disciplina.grid(row=1, column=0)
    entry_nome_disciplina.grid(row=1, column=1)
    dropdown_cursos.grid(row=1, column=2)
    button_cadastrar_disciplina.grid(row=1, column=3)

    admin_window.mainloop()

# Janela para usuários comuns
def usuario_window(usuario_id):
    usuario_window = tk.Tk()
    usuario_window.title("Área do Usuário")

    def cadastrar_questao():
        enunciado_questao = entry_enunciado_questao.get()
        disciplina_id = combo_disciplinas.get()
        c.execute("INSERT INTO questoes (enunciado, disciplina_id) VALUES (?, ?)", (enunciado_questao, disciplina_id))
        conn.commit()
        messagebox.showinfo("Questão Cadastrada", "Questão cadastrada com sucesso!")

    label_enunciado_questao = tk.Label(usuario_window, text="Enunciado da Questão:")
    entry_enunciado_questao = tk.Entry(usuario_window)

    c.execute("SELECT * FROM disciplinas")
    disciplinas = c.fetchall()
    combo_disciplinas = tk.StringVar()
    combo_disciplinas.set(disciplinas[0][0])  # Definir o valor padrão
    dropdown_disciplinas = tk.OptionMenu(usuario_window, combo_disciplinas, *disciplinas)

    button_cadastrar_questao = tk.Button(usuario_window, text="Cadastrar Questão", command=cadastrar_questao)

    label_enunciado_questao.grid(row=0, column=0)
    entry_enunciado_questao.grid(row=0, column=1)
    dropdown_disciplinas.grid(row=0, column=2)
    button_cadastrar_questao.grid(row=0, column=3)

    usuario_window.mainloop()

# Iniciar o programa
login_window()
conn.close()



NameError: name 'conn' is not defined